In [1]:
import requests
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
import warnings
warnings.filterwarnings('ignore')
import time
import re
import pandas as pd
import csv
import datetime

In [2]:
# first connect to the driver
driver=webdriver.Chrome(r"E:\Surendra Data\DATA Scientist Course Detail\chromedriver.exe")

# opening the page on selenium automated chrome browser
url = f"https://www.booking.com/index.en-gb.html?aid=397646&label=yho748jc-1DCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQPoAQGIAgGoAgO4Ap3Y96YGwAIB0gIkY2NlMDIyZmItZDdlZi00MWVkLWE5YzUtNmIzNmE1NWI3MDU32AIE4AIB&sid=0cb28093647b47834b4f3a02e11acd13&keep_landing=1&sb_price_type=total&"
driver.get(url)
driver.maximize_window()

# Closing the popup
wait = WebDriverWait(driver, 10)
popup_element = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "c0528ecc22")))
if popup_element.is_displayed():
    close_button = popup_element.find_element(By.CLASS_NAME, "eedba9e88a")
    close_button.click()
    time.sleep(3)
    
#Scroll to the bottom of the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# clicking the cities tab
cities_tab=driver.find_element(By.XPATH,'//li[2]/button/span[@class="bui-tab__text"]')
cities_tab.click()

In [3]:
# Find all the cities
city_1 = driver.find_elements(By.XPATH, '//div[@class="wcu-seo-destination__title"]/a')
city_2 = driver.find_elements(By.XPATH, '//div[@class="wcu-seo-popular-city bui-spacer--large"]/a')

city_url = city_1 + city_2
city_url

# Initialize a list to store the city url
city_urls = []

for i in city_url:
    citys_url = i.get_attribute('href')
    city_urls.append(citys_url)

In [4]:
# Define the list of city names we're interested in
cities_given = ["Dubai", "London", "Kuala","Manchester", "Delhi", "Birmingham", "Berlin",
                      "Sydney","Melbourne","Paris", "Tokyo", "Toronto"]

# finding the url for cities_of_interest
# Initialize a list to store city URLs
cities_url = []

for url in city_urls:
    for city in cities_given:
        if re.search(rf"\b{city}\b", url, re.I):
            cities_url.append(url)
            break

# Print the filtered URLs
for urls in cities_url:
    print(urls)

https://www.booking.com/city/ae/dubai.en-gb.html?aid=397646&amp;label=yho748jc-1BCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQGIAgGoAgO4Ap3Y96YGwAIB0gIkY2NlMDIyZmItZDdlZi00MWVkLWE5YzUtNmIzNmE1NWI3MDU32AIF4AIB&amp;sid=813c349cd67bbb9274c5c410902bf5e6&amp;lp_index2sr=1&amp;ilp=1
https://www.booking.com/city/gb/london.en-gb.html?aid=397646&amp;label=yho748jc-1BCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQGIAgGoAgO4Ap3Y96YGwAIB0gIkY2NlMDIyZmItZDdlZi00MWVkLWE5YzUtNmIzNmE1NWI3MDU32AIF4AIB&amp;sid=813c349cd67bbb9274c5c410902bf5e6&amp;lp_index2sr=1&amp;ilp=1
https://www.booking.com/city/my/kuala-lumpur.en-gb.html?aid=397646&amp;label=yho748jc-1BCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQGIAgGoAgO4Ap3Y96YGwAIB0gIkY2NlMDIyZmItZDdlZi00MWVkLWE5YzUtNmIzNmE1NWI3MDU32AIF4AIB&amp;sid=813c349cd67bbb9274c5c410902bf5e6&amp;lp_index2sr=1&amp;ilp=1
https://www.booking.com/city/gb/manchester.en-gb.html?aid=397646&label=yho748jc-1BCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQHoAQGIAgGoAgO4Ap3Y96YGwAIB0gIkY2NlMDIyZmItZDdlZ

As asked in quetion and also in project meeting, i have scrap from cities tab, and from cities tab only 11 cities (New delhi url is not there) url is coming. It was coming earlier but not now.

In [5]:
all_hotel_urls = []
for url in cities_url:
    driver.get(url)
    
    
    # Wait for the page to load successfully
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        
    # Fill the check-in & check-out fields, change the date which you need by yourself also
    check_in_date_input = driver.find_element(By.CSS_SELECTOR, '[aria-label="29 November 2023"]').click()
    check_out_date_input = driver.find_element(By.CSS_SELECTOR, '[aria-label="30 November 2023"]').click()
     
#   Click on search after giving check-in or out
    driver.find_element(By.XPATH, '//span[@class="e4adce92df"]').click()

    
    # Loop through multiple pages (scrapping up to 11 pages only for each city because everytime it is stuck and i am not able to get all due to issue faced)
    for page in range(10):
        # Closing the popup
        try:
            wait = WebDriverWait(driver, 5)
            popup_element = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "c0528ecc22")))
            if popup_element.is_displayed():
                close_button = popup_element.find_element(By.CLASS_NAME, "eedba9e88a")
                close_button.click()
                time.sleep(3)
        except TimeoutException:
            # Handle the case where the popup is not present
            pass
        
        try:
            url_elements = driver.find_elements(By.XPATH, '//div[@class="d6767e681c"]/h3/a')
            for element in url_elements:
                hotel = element.get_attribute('href')
                all_hotel_urls.append(hotel)
        
            try:
                next_button = driver.find_element(By.XPATH, '//div[@class="ef2dbaeb17"]/div[3]/button')
                next_button.click()
            except NoSuchElementException:
                # If the "Next" button is not found, break out of the loop
                break
        except StaleElementReferenceException:
            # Handle StaleElementReferenceException by re-locating the element
            continue

In [6]:
len(all_hotel_urls)

2750

In [7]:
# Create an empty list to store the dictionaries for each hotel
hotel_data_list = []

for i in all_hotel_urls:
    driver.get(i)
    time.sleep(5)
    
    # Create a dictionary to store data for the current hotel
    hotel_data = {}
    
    try:
        hotel = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        hotel_data['Hotel_name'] = hotel.text
    except NoSuchElementException:
        hotel_data['Hotel_name'] = '-'
        
    try:
        city = driver.find_element(By.XPATH, '//div[@class="e000754250"]/input')
        hotel_data['City'] = city.get_attribute("value")
    except NoSuchElementException:
        hotel_data['City'] = '-'
    
    try:
        loc = driver.find_element(By.XPATH, '//p[@class="address address_clean"]/span[1]')
        hotel_data['Location'] = loc.text
    except NoSuchElementException:
        hotel_data['Location'] = '-'
        
    try:
        rev = driver.find_element(By.XPATH,'//div[@class="daaa8ff09f f62fdb6838"]/div/div[2]/span[2]')
        hotel_data['Reviews'] = rev.text.split()[1]
    except NoSuchElementException:
        hotel_data['Reviews'] = '-'
        
    try:
        user = driver.find_element(By.XPATH,'//div[@class="daaa8ff09f f62fdb6838"]/div/div[2]/span[1]')
        hotel_data['Review_score'] = user.text
    except NoSuchElementException:
        hotel_data['Review_score'] = "-"
        
    try:
        star = driver.find_elements(By.XPATH, '//span[@class="fcd9eec8fb d31eda6efc c25361c37f"]')
        hotel_data['Star_rating'] = len(star)
    except NoSuchElementException:
        hotel_data['Star_rating'] = "-"
        
    try:
        score = driver.find_element(By.XPATH,'//div[@class="daaa8ff09f f62fdb6838"]/div/div[1]')
        hotel_data['User_rating'] = score.text
    except NoSuchElementException:
        hotel_data['User_rating'] = "-"
        
    try:
        wifi = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Free Wi-Fi")]')
        hotel_data['Free_wifi'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Free_wifi'] = 'No'
        
    try:
        room = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Family rooms")]')
        hotel_data['Family_rooms'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Family_rooms'] = 'No'
        
    try:
        smoke = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Non-smoking rooms")]')
        hotel_data['NonSmoking_rooms'] = 'Yes'
    except NoSuchElementException:
        hotel_data['NonSmoking_rooms'] = 'No'
        
    try:
        res = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Restaurent")]')
        hotel_data['Restaurent'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Restaurent'] = 'No'
        
    try:
        bar = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Bar")]')
        hotel_data['Bar'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Bar'] = 'No'
        
    try:
        heat = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Heating")]')
        hotel_data['Heating'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Heating'] = 'No'
        
    try:
        lift = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Lift")]')
        hotel_data['Lift'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Lift'] = 'No'
        
    try:
        food = driver.find_element(By.XPATH, '//h4[@class="ph-item-header"][contains(text(),"Breakfast")]')
        
        # Find the elements matching the XPath
        cuisine_elements = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
    
        # split the text
        cuisine_info = cuisine_elements.text.split()
        
        # Depending on the number of elements obtained after splitting, store them in the respective lists
        if len(cuisine_info) >= 3:
            hotel_data['Breakfast_Cuisine1'] = cuisine_info[0]
            hotel_data['Breakfast_Cuisine2'] = cuisine_info[1]
            hotel_data['Breakfast_Cuisine3'] = cuisine_info[2]
        elif len(cuisine_info) == 2:
            hotel_data['Breakfast_Cuisine1'] = cuisine_info[0]
            hotel_data['Breakfast_Cuisine2'] = cuisine_info[1]
            hotel_data['Breakfast_Cuisine3'] = '-'
        elif len(cuisine_info) == 1:
            hotel_data['Breakfast_Cuisine1'] = cuisine_info[0]
            hotel_data['Breakfast_Cuisine2'] = '-'
            hotel_data['Breakfast_Cuisine3'] = '-'
            
    except NoSuchElementException:
        hotel_data['Breakfast_Cuisine1'] = '-'
        hotel_data['Breakfast_Cuisine2'] = '-'
        hotel_data['Breakfast_Cuisine3'] = '-'
        
    try:
        room_type = driver.find_element(By.XPATH,'//span[@class="hprt-roomtype-icon-link "]')
        hotel_data['Room_type'] = room_type.text
    except NoSuchElementException:
        hotel_data['Room_type'] = '-'
        
    try:
        bed1 = driver.find_element(By.XPATH,'//div[@class="hprt-roomtype-bed"]/div/ul[contains(text(), "double")]')
        hotel_data['Double_bed'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Double_bed'] = 'No'
        
    try:
        bed2 = driver.find_element(By.XPATH,'//div[@class="hprt-roomtype-bed"]/div/ul[contains(text(), "single")]')
        hotel_data['Single_bed'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Single_bed'] = 'No'
        
    try:
        price = driver.find_element(By.XPATH, '//span[@class="prco-valign-middle-helper"]')
        hotel_data['Prices'] = price.text
    except NoSuchElementException:
        hotel_data['Prices'] = '-'
        
        
    # Append the hotel_data dictionary to the list
    hotel_data_list.append(hotel_data)

In [8]:
len(hotel_data_list)

2750

In [9]:
hotel_data_list[:3]

[{'Hotel_name': 'Days Hotel By Wyndham Dubai Deira',
  'City': 'Dubai',
  'Location': 'One Deira Plaza Gold Souk Metro Station Al Corniche - 111 , 119303 Dubai, United Arab Emirates',
  'Reviews': '4,049',
  'Review_score': 'Very good',
  'Star_rating': 3,
  'User_rating': '8.1',
  'Free_wifi': 'No',
  'Family_rooms': 'Yes',
  'NonSmoking_rooms': 'Yes',
  'Restaurent': 'No',
  'Bar': 'No',
  'Heating': 'No',
  'Lift': 'No',
  'Breakfast_Cuisine1': 'Continental,',
  'Breakfast_Cuisine2': 'Italian,',
  'Breakfast_Cuisine3': 'Full',
  'Room_type': 'Superior Queen Room, City View, 25% off Spa',
  'Double_bed': 'No',
  'Single_bed': 'No',
  'Prices': '₹ 10,069'},
 {'Hotel_name': 'Four Points by Sheraton Sheikh Zayed Road',
  'City': 'Dubai',
  'Location': 'Sheikh Zayed Road, Trade Centre, Dubai, United Arab Emirates',
  'Reviews': '2,926',
  'Review_score': 'Very good',
  'Star_rating': 4,
  'User_rating': '8.4',
  'Free_wifi': 'No',
  'Family_rooms': 'Yes',
  'NonSmoking_rooms': 'Yes',
  '

In [10]:
%store hotel_data_list

Stored 'hotel_data_list' (list)


In [11]:
# Assuming your data is stored in the 'hotel_data_list' variable

# Create DataFrames from the data
data1_columns = ["Sno", "Hotel Name", "City", "Location", "Number of Reviews", "User Ratings", "Star Rating", "Review Score"]
data1_data = [(i+1, d["Hotel_name"], d["City"], d["Location"], d["Reviews"], d["User_rating"], d["Star_rating"], d["Review_score"]) for i, d in enumerate(hotel_data_list)]
df1 = pd.DataFrame(data1_data, columns=data1_columns)

data2_columns = ["Sno", "Hotel Name", "Freewifi", "Family Rooms", "Non-Smoking Rooms", "Restaurant", "Bar", "Heating", "Lift", "Breakfast Cuisine1", "Breakfast Cuisine2", "Breakfast Cuisine3"]
data2_data = [(i+1, d["Hotel_name"], d["Free_wifi"], d["Family_rooms"], d["NonSmoking_rooms"], d["Restaurent"], d["Bar"], d["Heating"], d["Lift"], d["Breakfast_Cuisine1"], d["Breakfast_Cuisine2"], d["Breakfast_Cuisine3"]) for i, d in enumerate(hotel_data_list)]
df2 = pd.DataFrame(data2_data, columns=data2_columns)

data3_columns = ["Sno", "Hotel Name", "Room Type", "Doublebed", "Singlebed", "Prices"]
data3_data = [(i+1, d["Hotel_name"], d["Room_type"], d["Double_bed"],  d["Single_bed"], d["Prices"]) for i, d in enumerate(hotel_data_list)]
df3 = pd.DataFrame(data3_data, columns=data3_columns)

In [12]:
# Remove the currency symbol from the 'Prices' column
df3['Prices'] = df3['Prices'].str.replace('₹', '').str.replace(',', '')

In [13]:
# Specify the file path to save the CSV file
path1 = 'E:\Surendra Data\DATA Scientist Course Detail\Internship\Capstone Project\Data\Data1.csv'
path2 = 'E:\Surendra Data\DATA Scientist Course Detail\Internship\Capstone Project\Data\Data2.csv'
path3 = 'E:\Surendra Data\DATA Scientist Course Detail\Internship\Capstone Project\Data\Data3.csv'

# Save the DataFrame to a CSV file
df1.to_csv(path1, index=False)
df2.to_csv(path2, index=False)
df3.to_csv(path3, index=False)